In [8]:
#set up autoreload
%load_ext autoreload
%autoreload 2

import polars as pl
from pathlib import Path

pl.Config(set_fmt_float="full")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
path_to_dir = "/Users/claireboyd/internships/mansueto/firstamerican-etl/"
merged = pl.scan_parquet(
    path_to_dir+f"dev/national/merged.parquet"
    )
#     .with_columns([
#     pl.when(
#         pl.col('AssdTotalValue').is_null() | pl.col('MarketTotalValue').is_null() | pl.col('MarketTotalValue') == 0
#         ).then(None
#     ).otherwise(
#         pl.col('AssdTotalValue') / pl.col('MarketTotalValue')
#     ).alias('sales_ratio')
# ])

In [13]:
# sales_ratio_summary = merged.filter(
#     pl.col("sales_ratio").is_not_null()
# ).group_by(
#     "FIPS", "Year"
# ).agg(
#     pl.len().alias('sales_ratio_non_nulls_count'),
#     # pl.mean('sales_ratio').alias('sales_ratio_mean'),
#     # pl.median('sales_ratio').alias('sales_ratio_median')
# ).collect(streaming=True)

# sales_ratio_summary

In [15]:
# national_summary = merged.group_by(
#     "FIPS", "Year"
# ).agg(
#     pl.count('PropertyID').alias('TotalSales'),
#     pl.mean('AssdTotalValue').alias('AssdTotalValue_mean'),
#     pl.median('AssdTotalValue').alias('AssdTotalValue_median'),
#     pl.mean('sales_ratio').alias('sales_ratio_mean'),
#     pl.median('sales_ratio').alias('sales_ratio_median')
# ).collect(streaming=True)

In [19]:
# joined_summary = national_summary.join(
#     other=sales_ratio_summary,
#     on=["FIPS", "Year"],
#     how="left"
# ).with_columns([
#     (pl.col("sales_ratio_non_nulls_count")/pl.col("TotalSales")).alias("freq_valid_sales_ratio")
# ]).sort(
#     ["FIPS", "Year"], descending=True
# )

# joined_summary.write_csv("firstamerican_nationalsummary.csv")

In [32]:
simple_summary = merged.group_by(
    "FIPS", "Year"
).agg(
    pl.count('PropertyID').alias('TotalSales')
).collect(streaming=True)

In [42]:
simple_summary.pivot(
    index="FIPS",
    columns="Year",
    values="TotalSales"
).write_csv("count_bycounty.csv")